In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40 kB 1.7 MB/s eta 0:00:011
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-py3-none-any.whl size=30114 sha256=8add6b6b4598d4c300f4814767ee1440b896ccacbd5ae05a428af27aa710a1a0
  Stored in directory: /home/nobu_yamaguchi/.cache/pip/wheels/6c/c9/9c/363182ea34a736dae336eeaf0dd4a7eec3c6a5afe32373e1fe
  Created wheel for py-params: filename=py_params-0.9.7-py3-none-any.whl size=7302 sha256=8ac0dcbc968ce8599110cba6d6bb23899f4465fb7eea4584186b35556c449a0a
  Stored in directory: /home/nobu_yamaguchi/.cache/pip/wheels/47/3d/2d/bbffcfd6b9f4b8b5cbf07e7520ac2676192fe9431240c13ee8
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=db82dd4d94cafc56d4ee6e075c4dc30fc1c29e662d0b70567d39fb5248e18166
  Stored in directory: /home/nobu_yamaguchi/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
Successfully built bert-for-tf2 py-params params-flow
You should consider upgrading via 

In [4]:
!pip install tensorflow_hub

     |████████████████████████████████| 101 kB 2.7 MB/s ta 0:00:011
You should consider upgrading via the '/home/nobu_yamaguchi/anaconda3/bin/python -m pip install --upgrade pip' command.


In [29]:
import tensorflow as tf

import tensorflow_hub as hub
import numpy as up
import pandas as pd
from tensorflow.keras import layers
import bert

In [30]:
import numpy as np
import pandas as pd
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

In [100]:
patent_abstract = pd.read_csv('patent_abstract_5000_filter.csv')
patent_abstract.shape

(5000, 32)

In [129]:
patent_abstract['quality_rank'].value_counts()

0    3182
1    1818
Name: quality_rank, dtype: int64

In [103]:
df = patent_abstract[['text', 'quality_rank']]

In [83]:
df.iloc[10]

text            The invention relates to a power reserve indic...
quality_rank                                                    1
Name: 77, dtype: object

In [104]:
df.shape

(5000, 2)

In [105]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [106]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [107]:
abstracts = []
sentences = list(df['text'])
for sen in sentences:
    abstracts.append(sen)

In [108]:
print(df.columns.values)

['text' 'quality_rank']


In [109]:
df.quality_rank.unique()

array([1, 0])

In [93]:
df.quality_rank.iloc[6]

1

In [114]:
y = df.quality_rank

In [110]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [111]:
def tokenize_abstracts(text_abstracts):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_abstracts))

In [112]:
tokenized_abstracts = [tokenize_abstracts(abstract) for abstract in abstracts]

Prerparing Data For Training

In [115]:
abstracts_with_len = [[abstract, y[i], len(abstract)]
                 for i, abstract in enumerate(tokenized_abstracts)]

In [116]:
np.random.shuffle(abstracts_with_len)

In [117]:
abstracts_with_len.sort(key=lambda x: x[2])

In [118]:
sorted_abstracts_labels = [(abstract_lab[0], abstract_lab[1]) for abstract_lab in abstracts_with_len]

In [119]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_abstracts_labels, output_types=(tf.int32, tf.int32))

In [120]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [121]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 34), dtype=int32, numpy=
 array([[ 1037,  2152,  2373, ...,     0,     0,     0],
        [ 4725,  1998, 14709, ...,     0,     0,     0],
        [ 1996,  2556, 11028, ...,     0,     0,     0],
        ...,
        [ 3001,  1998,  4725, ...,  4263,  1012,     0],
        [ 1996,  2556, 11028, ...,  2966,  5097,  1012],
        [17770,  4140,  4360, ..., 10014,  4475,  1012]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 1], dtype=int32)>)

In [122]:
TOTAL_BATCHES = np.math.ceil(len(sorted_abstracts_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 20
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [123]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [159]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.4

NB_EPOCHS = 5

In [160]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [161]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["acc"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [162]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
150/150 [==============================] - 34s 227ms/step - loss: 0.6605 - acc: 0.6340
Epoch 2/5
150/150 [==============================] - 33s 222ms/step - loss: 0.6281 - acc: 0.6428
Epoch 3/5
150/150 [==============================] - 34s 225ms/step - loss: 0.4479 - acc: 0.7946
Epoch 4/5
150/150 [==============================] - 33s 222ms/step - loss: 0.1296 - acc: 0.9623
Epoch 5/5
150/150 [==============================] - 34s 224ms/step - loss: 0.0301 - acc: 0.9950


In [163]:
results = text_model.evaluate(test_data)
print(results)

      7/Unknown - 0s 37ms/step - loss: 1.2541 - acc: 0.6205[1.254145315715245, 0.62053573]
